# Experiment with the dvd rental database

In [1]:
%run connection.ipynb

Connecting with connection string : postgresql://postgres:letmein@db:5432/dvdrental
 * postgresql://postgres:***@db:5432/dvdrental
1 rows affected.
 * postgresql://postgres:***@db:5432/dvdrental
1 rows affected.
 * postgresql://postgres:***@db:5432/dvdrental
15 rows affected.


### What are the payments received in February where min payment is above $1?

In [2]:
%%sql

SELECT
    p.payment_id,
    p.customer_id,
    p.amount,
    p.payment_date
FROM payment p
WHERE EXTRACT (month FROM p.payment_date) = 2
    AND p.amount > 1
    
ORDER BY 2 DESC, 3 ASC

 * postgresql://postgres:***@db:5432/dvdrental
1625 rows affected.


,payment_id,customer_id,amount,payment_date
0,18464,599,1.99,2007-02-18 04:58:19.996577
1,18466,599,4.99,2007-02-21 14:03:04.996577
2,18465,599,6.99,2007-02-20 11:07:01.996577
3,18467,599,6.99,2007-02-21 17:14:31.996577
4,18463,598,2.99,2007-02-20 08:38:55.996577
...,...,...,...,...
1620,18501,1,3.99,2007-02-21 04:53:11.996577
1621,18499,1,4.99,2007-02-18 07:10:14.996577
1622,18498,1,4.99,2007-02-16 13:47:23.996577
1623,18495,1,5.99,2007-02-14 23:22:38.996577


### How much each customer spent on renting in total?

In [3]:
%%sql

SELECT
    p.customer_id,
    SUM(p.amount)
FROM payment p

GROUP BY 1
ORDER BY 2 DESC

 * postgresql://postgres:***@db:5432/dvdrental
599 rows affected.


,customer_id,sum
0,148,211.55
1,526,208.58
2,178,194.61
3,137,191.62
4,144,189.60
...,...,...
594,110,49.88
595,320,47.85
596,248,37.87
597,281,32.90


### Has all inventory ever been rented?

In [4]:
%%sql

SELECT
    f.film_id, f.title,
    i.store_id, i.inventory_id,
    COUNT(distinct r.rental_id) as rentals
FROM film f
    LEFT JOIN inventory i ON i.film_id = f.film_id
    LEFT JOIN rental r ON r.inventory_id = i.inventory_id

GROUP BY 1,2,3,4
ORDER BY 3 NULLS FIRST

 * postgresql://postgres:***@db:5432/dvdrental
4623 rows affected.


,film_id,title,store_id,inventory_id,rentals
0,332,Frankenstein Stranger,NaN,NaN,0
1,669,Pearl Destiny,NaN,NaN,0
2,108,Butch Panther,NaN,NaN,0
3,192,Crossing Divorce,NaN,NaN,0
4,128,Catch Amistad,NaN,NaN,0
...,...,...,...,...,...
4618,999,Zoolander Fiction,2.0,4572.0,4
4619,999,Zoolander Fiction,2.0,4573.0,5
4620,1000,Zorro Ark,2.0,4578.0,3
4621,1000,Zorro Ark,2.0,4579.0,5


### Find a customers first rental and various attributes about it?

In [5]:
%%sql

SELECT
    r.customer_id,
    MIN(r.rental_id) AS first_rental_id,
    (
        SELECT r2.rental_date FROM rental r2 WHERE r2.rental_id = MIN(r.rental_id)
    )::date first_rental_date
FROM rental r

GROUP BY 1
ORDER BY 1;


 * postgresql://postgres:***@db:5432/dvdrental
599 rows affected.


,customer_id,first_rental_id,first_rental_date
0,1,76,2005-05-25
1,2,320,2005-05-27
2,3,435,2005-05-27
3,4,1297,2005-06-15
4,5,731,2005-05-29
...,...,...,...
594,595,613,2005-05-28
595,596,303,2005-05-26
596,597,34,2005-05-25
597,598,3005,2005-06-20


### What are the atrributes of customers first order?

In [6]:
%%sql

SELECT
    p.*
FROM payment p
    INNER JOIN (
        SELECT p2.customer_id, MIN(p2.payment_date) as first_order_date
        FROM payment p2
        GROUP BY 1
    ) min_order ON min_order.first_order_date = p.payment_date

ORDER BY 2

 * postgresql://postgres:***@db:5432/dvdrental
600 rows affected.


,payment_id,customer_id,staff_id,rental_id,amount,payment_date
0,18495,1,1,1185,5.99,2007-02-14 23:22:38.996577
1,18502,2,1,2128,2.99,2007-02-17 19:23:24.996577
2,18503,3,1,1546,8.99,2007-02-16 00:02:31.996577
3,18507,4,1,1297,4.99,2007-02-15 07:59:54.996577
4,18513,5,1,1502,3.99,2007-02-15 20:31:40.996577
...,...,...,...,...,...,...
595,18456,595,2,1170,2.99,2007-02-14 22:16:01.996577
596,18458,596,1,1644,1.99,2007-02-16 07:26:44.996577
597,18460,597,1,2379,0.99,2007-02-18 13:28:05.996577
598,18463,598,1,3005,2.99,2007-02-20 08:38:55.996577


### How many customers purchese from multiple stores?

In [7]:
%%sql

SELECT
    t.customer_id,
    COUNT(*) FROM (
        SELECT r.customer_id, s.store_id
        FROM rental r
        LEFT JOIN staff s ON s.staff_id = r.staff_id
    ORDER BY 1
) t
GROUP BY 1;

 * postgresql://postgres:***@db:5432/dvdrental
599 rows affected.


,customer_id,count
0,1,32
1,2,27
2,3,26
3,4,22
4,5,38
...,...,...
594,595,30
595,596,28
596,597,25
597,598,22


### Return customers whose first order was on a weekend and worth over 5 and hoe have spent at least 100 in total.

In [8]:
%%sql

SELECT
    p.*,
    EXTRACT (dow FROM p.payment_date)::int,
    (
        SELECT SUM(p3.amount) FROM payment p3
        WHERE p3.customer_id = p.customer_id
    ) as total_spent
FROM payment p
WHERE p.payment_id = (
    SELECT MIN(p2.payment_id)
    FROM payment p2
    WHERE p2.customer_id = p.customer_id
)
-- Sunday 0, Saturday 6
AND EXTRACT (dow FROM p.payment_date)::int IN (0,6)
AND p.amount > 5

GROUP BY 1

HAVING (
    SELECT SUM(p3.amount) FROM payment p3
    WHERE p3.customer_id = p.customer_id
) > 100;

 * postgresql://postgres:***@db:5432/dvdrental
17 rows affected.


,payment_id,customer_id,staff_id,rental_id,amount,payment_date,date_part,total_spent
0,17509,342,2,2190,5.99,2007-02-17 23:58:17.996577,6,130.68
1,17526,346,1,1994,5.99,2007-02-17 09:35:32.996577,6,145.70
2,17866,436,1,2291,9.99,2007-02-18 06:05:12.996577,0,126.73
3,18099,497,2,2180,8.99,2007-02-17 23:16:09.996577,6,121.73
4,18367,572,2,1889,10.99,2007-02-17 02:33:38.996577,6,100.76
5,18395,579,2,2425,5.99,2007-02-18 16:06:11.996577,0,111.73
6,18446,593,2,2055,5.99,2007-02-17 13:55:29.996577,6,101.76
7,18554,16,2,1934,6.99,2007-02-17 05:33:23.996577,6,109.75
8,18572,21,2,2235,7.99,2007-02-18 02:37:16.996577,0,146.68
9,18636,32,2,1887,6.99,2007-02-17 02:21:44.996577,6,112.74


### How many of films are above and below the average replacement cost?

In [9]:
%%sql

SELECT
    t.grouping,
    COUNT(*) FROM (
        SELECT 'above' AS grouping, f.* FROM film f
        WHERE f.replacement_cost > (
            SELECT AVG(f2.replacement_cost) FROM film f2
        )

        UNION 

        SELECT 'below_eq' as grouping, f.* FROM film f
        WHERE f.replacement_cost <= (
            SELECT AVG(f2.replacement_cost) FROM film f2
        )
    ) t

GROUP BY 1


 * postgresql://postgres:***@db:5432/dvdrental
2 rows affected.


,grouping,count
0,above,536
1,below_eq,464


### Get customers total rental amount, but also their total in the month of their first order.

In [10]:
%%sql

WITH base_table AS (
    SELECT p.customer_id, SUM(p.amount) as total_spendings, (
        SELECT EXTRACT (MONTH FROM MIN(p2.payment_date)) FROM payment p2
        WHERE p2.customer_id = p.customer_id
    )::int as first_order
    FROM payment p
    GROUP BY 1
)

SELECT
    bt.*,
    (
        SELECT SUM(p3.amount) FROM payment p3
        WHERE p3.customer_id = bt.customer_id
        AND EXTRACT(MONTH FROM p3.payment_date) = bt.first_order
    ) as rental_amount_first_month
FROM base_table bt

 * postgresql://postgres:***@db:5432/dvdrental
599 rows affected.


,customer_id,total_spendings,first_order,rental_amount_first_month
0,184,80.80,2,3.98
1,87,137.72,2,25.95
2,477,106.79,2,25.96
3,273,130.72,3,62.87
4,550,151.69,2,39.94
...,...,...,...,...
594,449,80.83,2,8.97
595,64,91.70,2,3.98
596,520,127.69,2,12.97
597,55,84.81,2,18.95


### What are the top movies (rental amount) by rating?

In [11]:
%%sql

WITH nc17 AS (
    SELECT f.film_id, f.title, f.rating, SUM(p.amount)
    FROM film f
        JOIN inventory i ON f.film_id = i.film_id
        JOIN rental r ON r.inventory_id = i.inventory_id
        JOIN payment p ON p.rental_id = r.rental_id
    WHERE f.rating = 'NC-17'
    GROUP BY 1,2,3
    ORDER BY SUM(p.amount) DESC
    LIMIT 1
),
r AS (
    SELECT f.film_id, f.title, f.rating, SUM(p.amount)
    FROM film f
        JOIN inventory i ON f.film_id = i.film_id
        JOIN rental r ON r.inventory_id = i.inventory_id
        JOIN payment p ON p.rental_id = r.rental_id
    WHERE f.rating = 'R'
    GROUP BY 1,2,3
    ORDER BY SUM(p.amount) DESC
    LIMIT 1
),
g AS (
    SELECT f.film_id, f.title, f.rating, SUM(p.amount)
    FROM film f
        JOIN inventory i ON f.film_id = i.film_id
        JOIN rental r ON r.inventory_id = i.inventory_id
        JOIN payment p ON p.rental_id = r.rental_id
    WHERE f.rating = 'G'
    GROUP BY 1,2,3
    ORDER BY SUM(p.amount) DESC
    LIMIT 1
)

SELECT * FROM nc17
UNION
SELECT * FROM r
UNION
SELECT * FROM g


 * postgresql://postgres:***@db:5432/dvdrental
3 rows affected.


,film_id,title,rating,sum
0,764,Saturday Lambs,G,190.74
1,1000,Zorro Ark,NC-17,199.72
2,938,Velvet Terminator,R,152.77
